In [1]:
#| default_exp vars

In [2]:
#| hide
from nbdev.showdoc import *
import numpy as np

In [3]:
#| export
from fastcore.test import *
from fastcore.utils import *

import pandas as pd

## Formatação das datas

In [4]:
#|  hide
# Função para verificar o tamanho dos valores na coluna
def verifica_tamanho(valores, n):
    return all(len(str(valor)) == n for valor in valores)

In [5]:
#|  hide
# Função de ajuste para a data da safra e a de referência. 
# Podendo ser no formato de 6 dígitos YYYYMM ou no formato de 8 YYYYMMDD. 
# Para o formato YYYYMM vamos construir uma data como sendo no dia 01 de cada mês. 
# Já para datas no formato YYYYMMDD vamos utilizar a janela de tempo mais precisa fornecida pelo usuário.

def ajuste_tempo(df_, trns_time:str, ref_time:str):    

    if verifica_tamanho(df_[trns_time], n=6):
            # Cria datas com o formato YYYYMMDD para safra 
            df_['Safra_'] = df_[trns_time].apply(lambda x: str(x)+'01').apply(pd.to_datetime)
    elif verifica_tamanho(df_[trns_time], n=8):
            df_['Safra_'] = pd.to_datetime(df_[trns_time], format='%Y%m%d')
    else:
        print('Data da Safra no formato inválido!')

    if verifica_tamanho(df_[ref_time], n=6):
        # Cria datas com o formato YYYYMMDD para data de ref
        df_['ref_time_var_'] = df_[ref_time].apply(lambda x: str(x)+'01').apply(pd.to_datetime) 
    elif verifica_tamanho(df_[ref_time], n=8):
        df_['ref_time_var_'] = pd.to_datetime(df_[ref_time], format='%Y%m%d')
    else:
        print('Data de ref no formato inválido!')   


    return df_

## Função mode

In [6]:
#|  export
def mode(x):
    return pd.Series.mode(x).values[0]

Função mode para ser utilizada na criação de variáveis categóricas.

## Criação de variáveis

In [7]:
#| export
def create_vars_in_time(df_,id_cols,trns_time:str,ref_time:str,value_vars:list,window:list,operations:list):
    
    df = df_.copy()
    df_spine =  df[id_cols].drop_duplicates()
    df = ajuste_tempo(df, trns_time, ref_time)
    trns_time_var = "Safra_"
    ref_time_var = 'ref_time_var_'

    for var in value_vars:
        for size in window:
            time_cond = (df[trns_time_var] < df[ref_time_var]) & (df[trns_time_var] >= df[ref_time_var] + pd.DateOffset(months=size))
            df_grouped = df[time_cond].groupby(id_cols).agg({var:operations})
            df_grouped.columns = ['_'.join(col).strip()+'_'+str(abs(size))+'M' for col in df_grouped.columns.values]
            df_grouped.reset_index(inplace=True)
            df_spine = df_spine.merge(df_grouped,on=id_cols,how='left')
           
    return df_spine

Função utilizada para criar as variáveis categóricas e numéricas.

In [8]:
#| hide
import nbdev; nbdev.nbdev_export()